# Makemore pt.3 Kata no.2

## Setup and data

### Import packages

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

### Import data

In [2]:
words = open('../data/names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

### Get vocabulary and map to/from integers

In [3]:
chars = sorted(list(set(''.join(words))))
itos = {i+1:s for i, s in enumerate(chars)}
itos[0] = '.'
stoi = {s:i for i, s in itos.items()}
vocab_size = len(itos)

print(vocab_size)
print(itos)

27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


### Build dataset splits

In [4]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for name in words:
        context = [0] * block_size
        for char in name + '.':
            ix = stoi[char]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtst, Ytst = build_dataset(words[n2:])

torch.Size([182778, 3]) torch.Size([182778])
torch.Size([22633, 3]) torch.Size([22633])
torch.Size([22735, 3]) torch.Size([22735])


## Neural Network

### Layer modules

In [58]:
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out), generator=g) * 0.5
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([self.bias] if self.bias is not None else [])

class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # Parameters
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # Buffers
        self.running_mean = torch.ones(dim)
        self.running_var = torch.zeros(dim)

    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True, unbiased=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
            
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar

        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:

    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out

    def parameters(self):
        return []

### Neural Network initialisation

In [75]:
n_emb = 10
n_hidden = 100
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_emb), generator=g)
layers = [
    Linear((block_size * n_emb), n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
    Linear(            n_hidden, vocab_size), BatchNorm1d(vocab_size)
]

# Make last layer less confident
layers[-1].gamma *= 0.1
# Set gain for Linear layers
for layer in layers:
    if isinstance(layer, Linear):
        layer.weight *= 5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

47551


### Training loop

In [76]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

    # Build mini batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    # Forward pass
    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)

    # Backward pass
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()

    # Update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # Track stats
    if i % 10000 == 0:
        print(f'{i:7d} / {max_steps}: {loss}')
    lossi.append(loss.log10().item())

    # if i > 1000:
    #     break
        

      0 / 200000: 3.3162784576416016
  10000 / 200000: 2.4227488040924072
  20000 / 200000: 2.24312424659729
  30000 / 200000: 1.9483213424682617
  40000 / 200000: 2.5781843662261963
  50000 / 200000: 1.958975076675415
  60000 / 200000: 2.4463038444519043
  70000 / 200000: 2.1441855430603027
  80000 / 200000: 2.1744778156280518
  90000 / 200000: 2.0837926864624023
 100000 / 200000: 1.9194324016571045
 110000 / 200000: 2.0997962951660156
 120000 / 200000: 1.7937270402908325
 130000 / 200000: 2.016942262649536
 140000 / 200000: 2.3276352882385254
 150000 / 200000: 1.6029975414276123
 160000 / 200000: 2.4486570358276367
 170000 / 200000: 2.0809812545776367
 180000 / 200000: 2.4081408977508545
 190000 / 200000: 1.9214376211166382


## Diagnostics

### Calculate loss per split

In [77]:
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xtst, Ytst)
    }[split]

    emb = C[x]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, y)

    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.059532642364502
val 2.347982168197632
